In [1]:
import ipywidgets.widgets as widgets
from time import *
from adafruit_servokit import ServoKit
import traitlets
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import uuid
import subprocess
from jetcam.csi_camera import CSICamera

################################ clean folder ################################
subprocess.call(['rm', '-r','-f', 'snapshots'])
subprocess.call(['mkdir', '-p', 'snapshots'])
################################ clean folder done ################################


################################ setup camera ################################
image_widget = ipywidgets.Image(format='jpeg')

display(image_widget)


def saveData(steering,throttle):
    file_path = 'snapshots/' + str(uuid.uuid1()) +'_'+str(round(throttle,2))+'_'+str(round(steering,2))+ '_.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
    with open(file_path, 'wb') as f:
        f.write(image_widget.value)


def cameraCallback(change):
    image_widget.value = bgr8_to_jpeg(change['new'])
    if controller.axes[1].value!=0.0 or controller.axes[2].value!=0.0:
        saveData(controller.axes[2].value,controller.axes[1].value)
    

camera = CSICamera(width=328, height=246,capture_fps=60)
camera.running=True
################################ setup camera done ################################


################################ setup controller and servo ################################
kit = ServoKit(channels=16)
kit.servo[0].angle = 72
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)
################################ setup controller and servo done ################################



Image(value=b'', format='jpeg')

Controller()

In [2]:
camera.observe(cameraCallback,names='value')

def prepareEsc(change):
    print("电调自检验开始")
    if change['new']:
        print('press ')
        for i in range(50):
            kit.servo[1].angle = 90
            sleep(0.1)
    print("电调自检验完成")


def throttleControl(change):
    print('throttle changing')
    y=change['new']/(1.0/10.0)
    y=y*-1
    if y == -0.0:
        kit.servo[1].angle = 90
    else:
        kit.servo[1].angle = int(90+y)
        
def turnControl(change):
    print('stering changing')
    y=change['new']/(1.0/25.0)
    y=72+y
    kit.servo[0].angle = int(y)
    
def startLog(change):
    pass

def stopLog(change):
    pass
    
controller.buttons[0].observe(prepareEsc, names='value')
controller.axes[1].observe(throttleControl,names='value')
controller.axes[2].observe(turnControl,names='value')